# Identifying Port Calls in the PIERS BoL Data

### Problem Statement:

The PIERS data do not have clean arrival or departure dates; instead, cargo imported from a given shipping vessel at a given port is spread over multiple dates, often spanning several days up to several weeks, even though the vessels spend only a matter of hours at any given port call. The reason for this spread is not precisely known at this time, but given the underlying data come from US Customs, we expect it may be related to the time required to process the individual bills. 

Therefore we need to identify clusters of dates that indicate a unique port call for each vessel, with the goal of assigning the earliest date in the cluster as the actual arrival/departure date for imports/exports. 

Potential uses of this information include analyzing frequency of service at each port, identification of individual voyages, and many others. 

In [13]:
#prelims
import pandas as pd #v2.1.3
import polars as pl #v0.20.7
import plotly_express as px #v0.4.1 
import missingno as msno #v0.5.2
import datetime as dt
from sklearn.cluster import HDBSCAN

#display settings
pd.set_option('display.max_columns', None)

#enable string cache for polars categoricals
pl.enable_string_cache()

In [14]:
# basic import from data_cleaning notebook

#set paths
imports_path = 'data/raw_parquet/imports/'
exports_path = 'data/raw_parquet/exports/'
#init lazy dataframes
imports_lf = (
    pl.scan_parquet(imports_path+'*.parquet', parallel='columns')
    .with_columns([
        #create 2-digit hs code
        pl.col('hs_code').str.slice(0, length=2).alias('hs_2d'),
        #create year and month columns
        pl.col('date').dt.year().alias('year'),
        pl.col('date').dt.strftime('%Y%m').alias('month'),
        #convert zero volume values to null
        pl.col('teus').replace(0,None),
        pl.col('weight').replace(0,None),
        pl.col('qty').replace(0,None),
        #create direction column
        pl.lit('import').cast(pl.Categorical).alias('direction'), 
        #create lane_id
        (pl.col('departure_port_code').cast(pl.Utf8)+'_'+pl.col('arrival_port_code').cast(pl.Utf8))
        .cast(pl.Categorical)
        .alias('lane_id')
        ])
    #get lane name 
    .with_columns(
            pl.col('departure_port_name').drop_nulls().mode().first().over('lane_id').alias('best_departure_port_name'),
            pl.col('arrival_port_name').drop_nulls().mode().first().over('lane_id').alias('best_arrival_port_name')
        )
        .with_columns(
            (pl.col('best_departure_port_name').cast(pl.Utf8)+' — '+pl.col('best_arrival_port_name').cast(pl.Utf8))
            .str.to_titlecase()
            .cast(pl.Categorical)
            .alias('lane_name')
        )
        .drop('best_departure_port_name', 'best_arrival_port_name')
    #get most commonly used carrier name for each scac and vise-versa to normalize changes in names and codes 
    .with_columns(
        pl.col('carrier_name').drop_nulls().mode().first().over('carrier_scac')
        .alias('unified_carrier_name')
    )
    .with_columns(
        pl.col('carrier_scac').drop_nulls().mode().first().over('unified_carrier_name')
        .alias('unified_carrier_scac')
    )
    .with_columns(
        #create voyage_id from year, vessel IMO, voyage number, and origin country
        (pl.col('year').cast(pl.Utf8)+'_'+
         pl.col('vessel_id').cast(pl.Utf8)+'_'+
         pl.col('voyage_number').cast(pl.Utf8)+'_'+
         pl.col('origin_territory').cast(pl.Utf8))
        .cast(pl.Categorical).alias('voyage_id')
    )
)
exports_lf = (
    pl.scan_parquet(exports_path+'piers_exports_complete.parquet', parallel='columns')
    .with_columns([
        #create 2-digit hs code
        pl.col('hs_code').str.slice(0, length=2).alias('hs_2d'),
        #create year and month columns
        pl.col('date_departure').dt.year().alias('year'),
        pl.col('date_departure').dt.strftime('%Y%m').alias('month'),
        #convert zero volume values to null
        pl.col('teus').replace(0,None),
        pl.col('weight').replace(0,None),
        pl.col('qty').replace(0,None),
        #create direction column
        pl.lit('export').cast(pl.Categorical).alias('direction'),
        #create lane_id
        (pl.col('departure_port_code').cast(pl.Utf8)+'_'+pl.col('arrival_port_code').cast(pl.Utf8))
        .cast(pl.Categorical)
        .alias('lane_id')
        ])
    #get lane name 
    .with_columns(
            pl.col('departure_port_name').drop_nulls().mode().first().over('lane_id').alias('best_departure_port_name'),
            pl.col('arrival_port_name').drop_nulls().mode().first().over('lane_id').alias('best_arrival_port_name')
        )
        .with_columns(
            (pl.col('best_departure_port_name').cast(pl.Utf8)+' — '+pl.col('best_arrival_port_name').cast(pl.Utf8))
            .str.to_titlecase()
            .cast(pl.Categorical)
            .alias('lane_name')
        )
        .drop('best_departure_port_name', 'best_arrival_port_name')
    #get most commonly used carrier name and scac 
    .with_columns(
        pl.col('carrier_name').drop_nulls().mode().first().over('carrier_scac')
        .alias('unified_carrier_name')
    )
    .with_columns(
        pl.col('carrier_scac').drop_nulls().mode().first().over('unified_carrier_name')
        .alias('unified_carrier_scac')
    )
    .with_columns(
        #create voyage_id from year, vessel IMO, voyage number, and origin country
        (pl.col('year').cast(pl.Utf8)+'_'+
         pl.col('vessel_id').cast(pl.Utf8)+'_'+
         pl.col('voyage_number').cast(pl.Utf8)+'_'+
         pl.col('dest_territory').cast(pl.Utf8))
        .cast(pl.Categorical).alias('voyage_id')
    )
)

In [15]:
# function defs from data_cleaning

#fill nulls in volume cols with mean
def fill_volume(lf):
    '''ad hod function to fill volume columns with their means'''
    return (
        lf
        .with_columns([
            pl.col('teus').replace(0,None).fill_null(strategy='mean'),
            pl.col('weight').replace(0,None).fill_null(strategy='mean'),
            pl.col('qty').replace(0,None).fill_null(strategy='mean')
            ])
        )

#plotly graph inspecting nulls over time by group
def nulls_over_time_plotly(data_lf, group_var, time_var, value_var, title=False):
    '''
    Plots proportion of null values over time by group.
    INPUTS:
        data_lf - polars lazyframe containing the relevant data
        group_var - str - the name of the column by which to group
        time_var - str - the name of the time column (e.g., year, month) over which values will be counted
        value_var - str - the name of the column containing the variable in question
        title (default=False) - str - the title of the graph
    OUTPUT:
        a plotly express figure
    DEPENDS ON:
        polars
        plotly express 
    '''
    df = (
        #select relevant columns
        data_lf.select([group_var, time_var, value_var])
        #group by, creating null count and non-null count cols
        .group_by(group_var, time_var)
        .agg([pl.col(value_var).null_count().alias('null_count'),
                pl.col(value_var).count().alias('count')])
        #compute percent null and fill new column
        .with_columns((pl.col('null_count')/(pl.col('count')+pl.col('null_count'))).alias('null_percent'))
        #cast group col to string to allow sensible ordering of legend
        .cast({group_var:pl.Utf8})
        #sort by date (to allow proper visualization of lines) and group (for legend ordering) 
        .sort(time_var, group_var)
    ).collect()
    #plot
    fig = px.line(
        data_frame=df,
        x=time_var, y='null_percent',
        color=group_var,
        title= 'Count of nulls over time by source frame.' if not title else title
    )
    fig.show()
    del df

#plotly graph inspecting nulls by group
def nulls_by_group_plotly(data_lf, group_var, value_var, title=False):
    '''Plots proportion of null values in the given groups'''
    df = (
        data_lf.select([group_var, value_var])
        .group_by(group_var)
        .agg([pl.col(value_var).null_count().alias('null_count'),
                pl.col(value_var).count().alias('count')])
        .with_columns((pl.col('null_count')/(pl.col('count')+pl.col('null_count'))).alias('null_percent'))
        .cast({group_var:pl.Utf8})
        .sort('null_percent', descending=True)
    ).collect()
    #plot
    fig = px.bar(
        data_frame=df,
        x=group_var, y='null_percent',
        title= 'Null percent by group.' if not title else title
    )
    fig.show()

#fill nulls over groups given a single unique value per group
def fill_nulls_by_group(data_lf, group_vars, val_var):
    '''Fills null values by group if and only if the val_var for that group contains exactly one non-null unique value.
    INPUTS:
        data_lf - polars lazyframe containing the relevant data
        group_vars - iterable - the names of the columns by which groups will be created
        val_var - string - the name of the column in which nulls will be filled
    OUTPUT:
        filled_lf - the resultant lazyframe 
    DEPENDS ON:
        polars - current version written in polars 0.20.1
    '''
    filled_lf = (
        data_lf.with_columns(
            #if the group contains exactly one unique value: 
            pl.when(pl.col(val_var).drop_nulls().unique(maintain_order=True).len().over(group_vars)==1)
            #then fill the group with that value
            .then(pl.col(val_var).fill_null(pl.col(val_var).drop_nulls().unique(maintain_order=True).first().over(group_vars)))
            #otherwise do nothing
            .otherwise(pl.col(val_var))
            )
        )
    return filled_lf

def is_one2one(lf, col1, col2):
    '''checks if the two pl.LazyFrame columns constitute a 1-1 pairing'''
    forward = lf.group_by(col1).agg(pl.col(col2).drop_nulls().n_unique()).select(col2).max().collect().item() == 1
    back = lf.group_by(col2).agg(pl.col(col1).drop_nulls().n_unique()).select(col1).max().collect().item() == 1
    return (forward and back)

def count_unique_by_group(lf, group_vars, val_var):
    '''returns a dataframe of unique observations for the value variable over each group'''
    df = (
        lf.group_by(group_vars)
        .agg(
            pl.col(val_var).unique().alias('unique_values'),
            pl.col(val_var).n_unique().alias('n_unique')
        )
        .drop_nulls()
        .sort('n_unique', descending=True)
        .collect()
    )
    return df

def add_primary_carrier(lf):
    '''alternative ad hoc function to find primary carrier for each vessel and indicate cargo sharing'''
    lf = (
        lf.with_columns(
            pl.col('teus').sum()
            .over('vessel_id', 'month', 'unified_carrier_scac')
            .alias('sum_teus')
            )
        .with_columns(
            pl.col('unified_carrier_scac')
            .sort_by('sum_teus', descending=True)
            .drop_nulls().first()
            .over('vessel_id', 'month')
            .alias('vessel_owner')
            )
        #add bool col if bol is from primary carrier
        .with_columns(
            (pl.col('unified_carrier_scac')==pl.col('vessel_owner'))
            .alias('primary_cargo')
            )
    )
    return lf

def sharing_over_time_plotly(data_lf, group_var, include_missing_voyages=True, limit=10, title=False):
    '''
    Plots proportion of shared cargo over time (months) by group_var.
    INPUTS:
        data_lf - polars lazyframe containing the relevant data
        group_var - str - the name of the column by which to group
        include_missing_voyages - bool - default=True, when False, drops missing voyages
        title (default=False) - str - the title of the graph
    OUTPUT:
        a plotly express figure
    DEPENDS ON:
        polars
        plotly express 
    '''
    if not include_missing_voyages:
        df = data_lf.drop_nulls('voyage_id')
    else:
        df = data_lf
    
    df = (
        #select relevant columns
        df.select([group_var, 'month', 'primary_cargo', 'teus'])
        #sum teus over each group-month-shared 
        .group_by(group_var, 'month')
        .agg(
            (pl.col('teus')*pl.col('primary_cargo')).sum().alias('total_primary'),
            pl.col('teus').sum().alias('total_teus')
        )
        #create proportion shared
        .with_columns((1-(pl.col('total_primary')/pl.col('total_teus'))).alias('prop_shared'))
        #cast group col to string to allow sensible ordering of legend
        .cast({group_var:pl.Utf8})
        #sort by date (to allow proper visualization of lines) and group (for legend ordering) 
        .sort('month')
    ).collect()

    #limit categories
    top_groups = (
        data_lf.group_by(group_var)
        .agg(pl.col('teus').sum())
        .sort('teus', descending=True)
        .select(group_var)
        .limit(limit)
        .collect()
        .to_series()
        .cast(pl.Utf8)
    )
    
    #plot
    fig = px.line(
        data_frame=df.filter(pl.col(group_var).is_in(top_groups)).with_columns(pl.col('month').str.to_datetime('%Y%m')),
        x='month', y='prop_shared',
        color=group_var,
        title= 'Proportion of shared cargo over time.' if not title else title,
        labels={
            'prop_shared':'Proportion of cargo from non-primary carrier',
            'month':'Month'
        }
    )
    fig.show()


In [16]:
# fill volumes and add primary carrier 
imports_lf = fill_volume(imports_lf)
exports_lf = fill_volume(exports_lf)   
#add primary carrier and shared cargo columns to lfs
imports_lf = add_primary_carrier(imports_lf)
exports_lf = add_primary_carrier(exports_lf)


In [17]:
df = (
    imports_lf
    .group_by('date', 'vessel_id', 'arrival_port_code')
    .agg(pl.col('teus').sum().alias('sum_teus'))
    .with_columns(
        (pl.col('vessel_id').cast(pl.Utf8)+'_'+pl.col('arrival_port_code').cast(pl.Utf8)).cast(pl.Categorical).alias('vessel_port_pair')
    )
    .drop_nulls(subset='vessel_port_pair')
    .sort('date')
    .collect()
)
df.describe()

statistic,date,vessel_id,arrival_port_code,sum_teus,vessel_port_pair
str,str,f64,str,f64,str
"""count""","""1170725""",1.170725e6,"""1170725""",1.170725e6,"""1170725"""
"""null_count""","""0""",0.0,"""0""",0.0,"""0"""
"""mean""","""2014-10-29 01:…",9.1821e6,null,313.571474,null
"""std""",null,559181.745432,null,1062.148423,null
"""min""","""2005-01-01 00:…",0.0,null,0.02,null
"""25%""","""2008-12-27 00:…",9.146089e6,null,2.196793,null
"""50%""","""2015-07-06 00:…",9.292503e6,null,8.787171,null
"""75%""","""2020-06-12 00:…",9.416965e6,null,147.185107,null
"""max""","""2023-11-30 00:…",9.975363e6,null,154934.254373,null


In [18]:

samples = 100

pairs = df.select('vessel_port_pair').sample(samples).to_series()

pairs_df = pl.DataFrame({})

for i in range(samples):
    pair = pairs[i]
    #make single-column dataframe of dates where each date corresponds to a single TEU that arrived on that day 
    pair_1d = (
        df.filter(pl.col('vessel_port_pair')==pair)
        .select('date', pl.col('sum_teus').round())
        #explode dates by each teu 
        .select(pl.exclude('sum_teus').repeat_by('sum_teus').explode())
    )

    #find minimum number of occurances of a single date (needed for HDBSCAN param)
    min_sample = pair_1d.group_by('date').agg(pl.col('date').count().alias('count')).min().row(0)[1]

    #skip empty pairs
    if min_sample == 0:
        continue

    #instantiate clusterer
    clusterer = HDBSCAN(min_cluster_size=20, min_samples=min_sample) #we need to find a dynamic way of seleting these parameters
    #get clusters
    clusterer.fit(pair_1d)
    #add back to pair_1d
    pair_df = (
        pair_1d
        #add cluster column
        .with_columns(
            pl.Series(name='cluster', values=clusterer.labels_)
        )
        #add date_arrival column
        .with_columns(
                #when date matches the mode of each cluster
                pl.when(pl.col('date') == pl.col('date').mode().first().over('cluster'))
                #fill with that date, otherwise fill with null
                .then(pl.col('date'))
                .otherwise(pl.lit(None))
                #forward fill the arrival date to the mode of next cluster
                .forward_fill()
                #backward fill the first part of first cluster
                .backward_fill()
                #name column
                .alias('date_arrival')
            )
        #groupby date to simplify
        .group_by('date')
        .agg(pl.col('date_arrival').first())
        #add pair label
        .with_columns(pl.lit(pair).alias('vessel_port_pair').cast(pl.Categorical))
    )

    #init or concat pairs_df
    if i == 0:
        pairs_df = pair_df   
    else:
        pairs_df = pl.concat([pairs_df,pair_df], how='vertical') 

In [19]:
pairs_df.describe()

statistic,date,date_arrival,vessel_port_pair
str,str,str,str
"""count""","""6604""","""6604""","""6604"""
"""null_count""","""0""","""0""","""0"""
"""mean""","""2013-01-10 10:…","""2012-12-07 03:…",null
"""std""",null,null,null
"""min""","""2005-01-01 00:…","""2005-01-02 00:…",null
"""25%""","""2005-11-21 00:…","""2005-11-16 00:…",null
"""50%""","""2013-09-03 00:…","""2013-06-24 00:…",null
"""75%""","""2019-02-17 00:…","""2018-12-22 00:…",null
"""max""","""2023-11-29 00:…","""2023-11-29 00:…",null


In [20]:
test_lf = (
    imports_lf
    .with_columns((pl.col('vessel_id').cast(pl.Utf8)+'_'+pl.col('arrival_port_code').cast(pl.Utf8)).cast(pl.Categorical).alias('vessel_port_pair')))

In [21]:
pairs_lf = pairs_df.lazy()

joined_lf = (
    test_lf.join(pairs_lf, on=['date', 'vessel_port_pair'], how='left')
)

joined_df = (
    joined_lf
    .select('date', 'vessel_port_pair', 'bol_id', 'teus', 'date_arrival')
    .collect()
)

display(joined_df.head())
joined_df.describe()

date,vessel_port_pair,bol_id,teus,date_arrival
datetime[μs],cat,str,f64,datetime[μs]
2005-10-12 00:00:00,"""9191321_5201""","""WWLN_JP363426""",2.196793,null
2005-10-12 00:00:00,"""9143544_1703""","""LYKE_LDNAL0036…",2.196793,null
2005-09-16 00:00:00,"""8214152_1401""","""ACLO_5216S2381…",2.196793,null
2005-03-30 00:00:00,"""9155121_1601""","""MLSL_CMB300747…",2.196793,null
2005-08-29 00:00:00,"""9043782_2709""","""KLIN_SFLSA0016…",2.196793,null


statistic,date,vessel_port_pair,bol_id,teus,date_arrival
str,str,str,str,f64,str
"""count""","""178921553""","""167409519""","""178921553""",1.78921553e8,"""679258"""
"""null_count""","""0""","""11512034""","""0""",0.0,"""178242295"""
"""mean""","""2016-02-25 02:…",null,null,2.196795,"""2014-09-15 08:…"
"""std""",null,null,null,2.918122,null
"""min""","""2005-01-01 00:…",null,"""-1_CSHSE000999…",0.01,"""2005-01-02 00:…"
"""25%""","""2012-03-26 00:…",null,null,1.75,"""2008-12-16 00:…"
"""50%""","""2016-09-18 00:…",null,null,2.196793,"""2015-05-12 00:…"
"""75%""","""2020-09-06 00:…",null,null,2.196793,"""2019-07-23 00:…"
"""max""","""2023-11-30 00:…",null,"""zimu_ZIMUZHJ00…",1730.75,"""2023-11-29 00:…"


In [6]:
lf.select('vessel_port_pair').collect().to_series().n_unique()

176533

In [33]:
df = (
    imports_lf
    .group_by('date', 'vessel_id', 'arrival_port_code')
    .agg(
        pl.col('teus').sum().alias('sum_teus'),
        pl.col('bol_id').count().alias('num_bills')
    )
    .with_columns(
        (pl.col('vessel_id').cast(pl.Utf8)+'_'+pl.col('arrival_port_code').cast(pl.Utf8)).cast(pl.Categorical).alias('vessel_port_pair')
    )
    .sort('date')
    .collect()
)


df.head()

date,vessel_id,arrival_port_code,sum_teus,num_bills,vessel_port_pair
datetime[μs],i32,cat,f64,u32,cat
2005-01-01 00:00:00,9159115,"""4909""",65.903779,30,"""9159115_4909"""
2005-01-01 00:00:00,9278088,"""2704""",4.393585,2,"""9278088_2704"""
2005-01-01 00:00:00,9102289,"""2704""",4.393585,2,"""9102289_2704"""
2005-01-01 00:00:00,9286267,"""2811""",2.196793,1,"""9286267_2811"""
2005-01-01 00:00:00,9143104,"""4601""",2.196793,1,"""9143104_4601"""


In [34]:
date = dt.datetime(2023,1,1)

toppairs = (
    df
    .filter(pl.col('date')>date)
    .group_by('vessel_port_pair')
    .agg(pl.col('date').n_unique().alias('num_dates'))
    .drop_nulls()
    .sort('num_dates', descending=True)
    .limit(20)
    .select('vessel_port_pair')
    .to_series()
)

In [35]:
%%script echo skipping
for pair in toppairs:
    px.bar(
        df.filter(
            pl.col('vessel_port_pair')==pair,
            pl.col('date')>date
            ), 
        x='date', y='sum_teus', color='vessel_port_pair'
    ).show()

skipping


see if voyage id is unique-ish per cluster

In [58]:
toppair = toppairs[3]

pair_1d = (
    df.filter(
            pl.col('vessel_port_pair')==toppair,
            pl.col('date')>date
            )
    .select('date', pl.col('sum_teus').round())
    .select(pl.exclude('sum_teus').repeat_by('sum_teus').explode())
)

#px.histogram(pair_1d, nbins=365)

In [59]:
#instantiate clusterer
clusterer = HDBSCAN(min_cluster_size=20, min_samples=5)
#get clusters
clusterer.fit(pair_1d)
#add back to pair_1d
clustered_pair_id = (
    pair_1d.with_columns(
    pl.Series(name='cluster', values=clusterer.labels_)
    )
)


In [65]:
clustered_pair_id.with_columns(
    #when date matches the mode of each cluster
    pl.when(pl.col('date') == pl.col('date').mode().first().over('cluster'))
    #fill with that date, otherwise fill with null
    .then(pl.col('date'))
    .otherwise(pl.lit(None))
    #forward fill the arrival date to the mode of next cluster
    .forward_fill()
    #backward fill to capture the first part of first cluster
    .backward_fill()
    #name column
    .alias('date_arrival')
    )


date,cluster,date_arrival
datetime[μs],i64,datetime[μs]
2023-01-02 00:00:00,0,2023-01-10 00:00:00
2023-01-02 00:00:00,0,2023-01-10 00:00:00
2023-01-02 00:00:00,0,2023-01-10 00:00:00
2023-01-02 00:00:00,0,2023-01-10 00:00:00
2023-01-02 00:00:00,0,2023-01-10 00:00:00
2023-01-02 00:00:00,0,2023-01-10 00:00:00
2023-01-02 00:00:00,0,2023-01-10 00:00:00
2023-01-02 00:00:00,0,2023-01-10 00:00:00
2023-01-02 00:00:00,0,2023-01-10 00:00:00


In [66]:

px.histogram(data_frame=clustered_pair_id, x='date', color='cluster', nbins=365)

In [40]:
clustered_pair_id.group_by('cluster').agg(pl.col('date').n_unique().sort(descending=True)).head(20)

cluster,date
i64,u32
1,10
18,3
12,3
5,6
19,2
6,5
11,3
0,9
8,7


In [41]:
test_df = imports_lf.select('date').group_by('date').len().collect().limit(20)

In [42]:
what = test_df.select(pl.exclude('len').repeat_by('len').explode())

In [43]:
#ref


data = pd.read_csv('data_set.csv')

clusterer = hdbscan.HDBSCAN(min_cluster_size=4,min_samples=8)
clusterer.fit(data)
data['cluster'] = clusterer.labels_

fig = px.bar(data,x='price',y='quantity',color='cluster',orientation='v')
fig.show()

FileNotFoundError: [Errno 2] No such file or directory: 'data_set.csv'